In [24]:
# init
import os
import sys

from IPython.core.interactiveshell import InteractiveShell
import numpy as np
import pandas as pd
import plotly.graph_objs as go
from plotly import tools
import plotly.figure_factory as figf
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import synapseclient
from synapseclient import Activity, Schema, Table, as_table_columns
import synapseutils
from tqdm import tqdm, tqdm_pandas, tqdm_notebook

sys.path.insert(0, os.path.abspath('..'))

init_notebook_mode(connected=True)
InteractiveShell.ast_node_interactivity = 'all'
tqdm.pandas(leave=True)

syn = synapseclient.Synapse()
syn.login()

Welcome, Luke Waninger!



In [25]:
def fx(x):
    i = x.find('city')
    if i > 0:
        return x[:i].strip()
    
    i = x.find('town')
    if i > 0:
        return x[:i].strip()
    
    return x


fips_syn = syn.get('syn16975424')
fips = pd.read_csv(fips_syn.path)

fips.loc[fips.summary_level == 10,  'summary_level'] = 'country'
fips.loc[fips.summary_level == 40,  'summary_level'] = 'state'
fips.loc[fips.summary_level == 50,  'summary_level'] = 'county'
fips.loc[fips.summary_level == 162, 'summary_level'] = 'city'
fips.loc[fips.summary_level == 61,  'summary_level'] = 'county_subdivision'
fips.loc[fips.summary_level == 170, 'summary_level'] = 'unified_government'

fips.loc[fips.summary_level == 'county', 'name'] = fips.loc[fips.summary_level == 'county'].name.apply(lambda f: f[:f.find('County')].strip())
fips.loc[fips.summary_level=='city', 'name'] = fips.loc[fips.summary_level=='city'].name.apply(fx)

In [26]:
fips.head(3)

,summary_level,state_fips,county_fips,county_subdivision_fips,place_fips,city_fips,name
0,country,0,0,0,0,0,United States
1,state,1,0,0,0,0,Alabama
2,county,1,1,0,0,0,Autaug


In [27]:
fips_syn = syn.setProvenance(
    syn.store(Table(
        Schema(name='FIPS', columns=as_table_columns(fips), parent='syn16816579'), fips)
    ),
    activity=Activity(
        name='FIPS Processing',
        description='Make the FIPS data more human readable',
        used=[           
            dict(
                name='US Census Bureau',
                url='https://www.census.gov/geographies/reference-files/2016/demo/popest/2016-fips.html',
            )
        ]
    )
)